## Basic Algorithms Lab: Fibonacci and Insertion Sort


Another form of recursion is tree recursion. Consider computing a fibonacci sequence, in which each number is the sum of the previous two, with the first two taken to be 0 and 1.

### Recursive Fibonacci. 


We write Fibonacci recursively with the first two numbers as base cases. (as in the last lab)

![](https://mitpress.mit.edu/sicp/full-text/book/ch1-Z-G-13.gif)

(from SICP)

Signature: `def fib_recursive(n)`

In [1]:
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [1, 2, 7, 13, 29, 33]:
    print(fib_recursive(i))

0
1
8
144
317811
2178309


### Q1. 

What are the space and time complexities of this implementation? Is this implementation tail-recursive?


*your answer here*


Space complexity: 1 Frame for n<=2, 1 + num_frames(n-1) + num_frames(n-2) otherwise. This is the fibonacci sequence itself, so complexity is O(fibonacci(n))

Time complexity: Same as with space, O(1) to add them together + time complexity of (n-1) and (n-2) case, which again becomes the fibonacci equation

It is not tail-recursive, the return statement has recursive calls but they're added together so the current frame's environment is needed


### Dynamic Programming and Iteration

From Skiena
>..**dynamic programming**, which typically removes one element from the problem, solves the smaller problem, and then uses the solution to this smaller problem to add back the element in the proper way. **Divide-and-conquer** instead splits the problem in (say) halves, solves each half, then stitches the pieces back together to form a full solution.

>Dynamic programming is a technique for efficiently implementing a recursive algorithm by storing partial results. The trick is seeing whether the naive recursive algorithm computes the same subproblems over and over and over again. If so, storing the answer for each subproblems in a table to look up instead of recompute can lead to an efficient algorithm. Start with a recursive algorithm or definition. Only once we have a correct recursive algorithm do we worry about speeding it up by using a results matrix. Dynamic programming is generally the right method for optimization problems on combinatorial objects that have an inherent left to right order among components. nents. Left-to-right objects includes: character strings, rooted trees, polygons, and integer sequences.

### Q2.

Here is an implementation of Fibonacci using dynamic programming: they key is to notice that the recurrence we used can be put into an iterative form and just stored in an everr increasing array. What is the space and time complexity here?

In [2]:
def fib_iterative(n):
    fibs=[]
    fibs.append(0)
    fibs.append(1)
    for i in range(2, n):
         fibs.append(fibs[i-1]+fibs[i-2])
    return fibs[n-1]

In [3]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


*your answer here*

Space complexity - O(n) for the n-length list

Time complexity - Just looking at the previous 2 elements in list for every n and summing so O(n)


### Recursive Fibonacci with cacheing

In [4]:
def cache(f):
    """a single argument function whose values may be cached"""
    cache = {}
    def memoized_func(x):
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    memoized_func.__name__ = f.__name__
    return memoized_func
    

In [5]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        def inner(*args, **kwargs):
            n = func.__name__
            #your code here
            func_out = func(*args, **kwargs)
            if n in count_dictionary:
                count_dictionary[n] += 1
            else:
                count_dictionary[n] = 1
            return func_out
        return inner
    return call_counter_decorator

### Q3. 

Use `cache` and `call_counter` as a decorator on `fib_recursive` and print the fibonacci numbers for 7,13,29, 33 as per last time. What order should these decorators be called to make sure `call_counter` gets the actual number of calls to `fib_recursive`?

In [6]:
#your code here
numbers = [7,13,29,33]
counter = {}
@call_counter(counter)
@cache
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in numbers:
    counter['fib_recursive'] = 0
    fib_recursive(i)
    print(counter)

{'fib_recursive': 11}
{'fib_recursive': 13}
{'fib_recursive': 33}
{'fib_recursive': 9}


Not enough information about what the "actual number of calls" is defined as and if the counters should be running (so not reset to 0 after every completed call from notebook). If cache decorated above call_counter, the memo function from cache will first be hit, then the inner function from call_counter. This means if the value is in the cache, it will return and the call will never be counted so the counter is just referring to how many uncached values there are. In the example below, once something has been cached the counter never changes on subsequent calls. If those two are reversed then the initial call will be counted.

In [7]:
numbers = [5,5,5,5,5]
counter = {}
@cache
@call_counter(counter)
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)
for i in numbers:
    counter['fib_recursive'] = 0
    fib_recursive(i)
    print(counter)

{'fib_recursive': 5}
{'fib_recursive': 0}
{'fib_recursive': 0}
{'fib_recursive': 0}
{'fib_recursive': 0}


### Q4.

What is the time and space complexity of the the memoized Fibonacci? HINT: assume evaluation happens left to right on sub-expressions, so that `fib_recursive(n-1)` side of the tree is evaluated first, and thus the tree is evaluated depth first, from left to right. What kind of pruning happens in the tree?

*your answer here*

Space: Any right child of the tree is drawn from the cache, as it goes depth first along the leftmost call and caches everything - fib(n-1), fib(n-2), fib(n-3), etc. So O(n) frames needed

Time: There's n-1 calls to the leftmost child before everything is cached, then for every subframe (n-1 of them) there's a cache hit on the right child and a summation, then the final summation at the top so still O(n) altogether

### Q5.

Do you really need to store the entire array in the dynamic programming implementation? Isnt it enough to have only saved the previous two Fibonacci numbers? Implement such an algorithm in `fib_iterative2(n)`. What is its space and time complexity?

In [8]:
#your code here
def fib_iterative2(n):
    fibs=[0,1]
    for i in range(1, n):
        sum = fibs[0] + fibs[1]
        fibs[0] = fibs[1]
        fibs[1] = sum
    return fibs[0]

In [9]:
#your code here
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative2(i))

0
1
8
144
317811
2178309


*your answer here*

The space is just the 2 array elements so O(1), time is still the same as before

### Q6.

Write an algorithm for insertion sort. 

![](https://upload.wikimedia.org/wikipedia/commons/0/0f/Insertion-sort-example-300px.gif)

(from wikipedia)

The algorithm is also illustrated here: http://cs.armstrong.edu/liang/animation/web/InsertionSort.html and may be described thus

insertion sort is a method for sorting that starts with a single element (thus forming a trivially sorted list) and then incrementally inserts the remaining elements so that the list stays sorted.

Talk about the best, worst and average complexity of insertion sort. Use the same list A from the lecture
`A=[5,2,1,3,8,6,9]`

In [10]:
#your code here
def insertion_sort(l):
    for i in range(1, len(l)):
        temp = l[i]
        j = i-1
        while temp < l[j] and j >= 0:
            l[j+1] = l[j]
            j -= 1
        l[j+1] = temp
    return l

In [11]:
#your code here
print(insertion_sort([5,2,1,3,8,6,9]))

[1, 2, 3, 5, 6, 8, 9]


*your answer here*

best case it's sorted so you never have to go through the while loop O(n)

worst case it's in reverse order so for every i you have to loop through i times which becomes O(n^2)

on average, similar to the selection sort, it's about n^2/2 comparisons so still O(n^2)